## Lab Assignment 2
### Data Mining 7331 Section 403
---
- Brian Coari
- Stephen Merritt
- Cory Thigpen
- Quentin Thomas

### Data Preparation

#### Section 1

- (10pts) Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

---

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv('C:/Users/william.thigpen/Documents/olympics/data/athletes_cleaned_merged.csv')
df.head()

,Unnamed: 0,NOC,Country,ID,Name,Sex,Age,Height,Weight,Team,Games,Year,Season,City,Sport,Event,Medal
0,872,AFG,Afghanistan,502.0,Ahmad Shah Abouwi,M,25,176,73,Afghanistan,1956 Summer,1956,Summer,Melbourne,Hockey,Hockey,No Medal
1,1950,AFG,Afghanistan,1076.0,Jammal-ud-Din Affendi,M,28,176,73,Afghanistan,1936 Summer,1936,Summer,Berlin,Hockey,Hockey,No Medal
2,1992,AFG,Afghanistan,1101.0,Mohammad Anwar Afzal,M,23,177,73,Afghanistan,1948 Summer,1948,Summer,London,Football,Football,No Medal
3,3129,AFG,Afghanistan,1745.0,Mohammad Aktar,M,17,156,48,Afghanistan,1980 Summer,1980,Summer,Moskva,Wrestling,"Light-Flyweight, Freestyle",No Medal
4,8410,AFG,Afghanistan,4628.0,Mohammad Daoud Anwary,M,22,172,76,Afghanistan,1964 Summer,1964,Summer,Tokyo,Wrestling,"Bantamweight, Freestyle",No Medal


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266954 entries, 0 to 266953
Data columns (total 17 columns):
Unnamed: 0    266954 non-null int64
NOC           266954 non-null object
Country       266954 non-null object
ID            266954 non-null float64
Name          266954 non-null object
Sex           266954 non-null object
Age           266954 non-null int64
Height        266954 non-null int64
Weight        266954 non-null int64
Team          266954 non-null object
Games         266954 non-null object
Year          266954 non-null int64
Season        266954 non-null object
City          266954 non-null object
Sport         266954 non-null object
Event         266954 non-null object
Medal         266954 non-null object
dtypes: float64(1), int64(5), object(11)
memory usage: 34.6+ MB


The intial data set contained the following data of different types:

| Nominal | Ordinal | Numeric | Binary |
| ------- | ------- | ------- | ------ |
| Games   | City    | Age     |        |
| Name    | Event   | Height  |        |
| NOC     | ID      | Weight  |        |
| Season  | Medal   | Year    |        |
| Sex     |         |         |        |
| Team    |         |         |        |

To prepare the lab, we brought population data into the data set and changed all National Olympic Committee or `NOC` data points to match their corresponding `Country`. For countries without entries in the worldbank URL we hard-coded the populations. There were 106 observations in the data set not associated with a country, so we dropped them. These were `Refugee Athletes` and `Individual Olympic Athletes`. `Sex`, `Season`, and `Medal` were converted to numeric factors and `ID`, `NOC`, `Name`, `Team`, and `Games` were dropped due to lack of relevancy.

`Country`, `Sport`, and `Event` had too many variables for binary coding, so one-hot encoding was performed.

#### Section 2

- (5pts) Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

---

Three variables were added to the data set: `Population_Prop`, `BMI`, and `Previous Medals`. The `Population_Prop` variable was added to represent the percentage of the overall world population for a given `Country`. Athlete `BMI` was added to the dataframe where $\omega$ represents `Weight` in kilograms, and $\eta$ represents `Height` in centimeters, and $C$ is a constant at `10,000`:

$$
BMI = \left(\frac{\omega}{\eta \eta}\right) C
$$

`Previous_Medals` was created as a binary representation if an athlete had won a medal at a previous olympic games (indicated by a `1`). The `Medal` variables was converted from ordinal to binary, where `Gold`, `Silver`, or `Bronze` were given a `1` and `No Medal` performers were given a `0`. `Season` and `Sex` were also converted to binary. 

One-hot encoding was performed for binarization of the remaining nominal variables(`Country`, `Event`, and `Sport`) so they could be included as features to train the classification model.

The final data set contains the following data of different types:

| Nominal | Ordinal | Numeric         | Binary          |
| ------- | ------- | --------------- | --------------- | 
|         |         | Age             | Country         |
|         |         | BMI             | Event           |
|         |         | Height          | Medal           |
|         |         | Population_Prop | Previous-Medals |
|         |         | Weight          | Season          |
|         |         | Year            | Sex             |
|         |         |                 | Sport           |

In [7]:
#Combine existing data with population data from:
#https://data.world/worldbank/total-population-per-country/file/POP_TOTAL_DS2_en_v2.csv
df_pop = pd.read_csv('C:/Users/william.thigpen/Documents/olympics/data/POP_TOTAL.csv', encoding = "ISO-8859-1")
df_pop = df_pop[['Country Code', '2015']]
#Change all Russia NOCs
df['NOC'] = np.where(df['NOC'] == 'EUN', 'RUS', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'URS', 'RUS', df['NOC'])
#Change all Australia NOCs
df['NOC'] = np.where(df['NOC'] == 'ANZ', 'AUS', df['NOC'])
#Change all German NOCs
df['NOC'] = np.where(df['NOC'] == 'FRG', 'GER', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'GDR', 'GER', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'SAA', 'GER', df['NOC'])
#Change all Congo NOCs
df['NOC'] = np.where(df['NOC'] == 'CGO', 'COD', df['NOC'])
#Change all Czech NOCs
df['NOC'] = np.where(df['NOC'] == 'BOH', 'CZE', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'TCH', 'CZE', df['NOC'])
#Change all Yemen NOCs
df['NOC'] = np.where(df['NOC'] == 'YAR', 'YEM', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'YMD', 'YEM', df['NOC'])
#Change all Greek NOCs
df['NOC'] = np.where(df['NOC'] == 'CRT', 'GRE', df['NOC'])
#Change all Zimbabwe NOCs
df['NOC'] = np.where(df['NOC'] == 'RHO', 'ZIM', df['NOC'])
#Change all Malaysia NOCs
df['NOC'] = np.where(df['NOC'] == 'MAL', 'MAS', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'NBO', 'MAS', df['NOC'])
#Change all Vietnam NOCs
df['NOC'] = np.where(df['NOC'] == 'VNM', 'VIE', df['NOC'])
#Change all Trinidad and Tobego NOCs
df['NOC'] = np.where(df['NOC'] == 'WIF', 'TTO', df['NOC'])
#Change all Trinidad and Tobego NOCs
df['NOC'] = np.where(df['NOC'] == 'UAR', 'SYR', df['NOC'])
#Change all Serbian NOCs
df['NOC'] = np.where(df['NOC'] == 'SCG', 'YUG', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'SRB', 'YUG', df['NOC'])
#Change all Canada NOCs
df['NOC'] = np.where(df['NOC'] == 'NFL', 'CAN', df['NOC'])
df_pop.rename(index = str, columns = {'Country Code' : 'NOC', '2015': '2015 Population'}, inplace = True)
df_pop_merge = pd.merge(left = df, right = df_pop, how = 'left', on=['NOC'], left_index=False)
df_pop_merge.loc[df_pop_merge['Country'] == 'Taiwan', '2015 Population'] =  23485755
df_pop_merge.loc[df_pop_merge['Country'] == 'Saint Vincent', '2015 Population'] =  109643
df_pop_merge.loc[df_pop_merge['Country'] == 'Palestine', '2015 Population'] =  4817000
df_pop_merge.loc[df_pop_merge['Country'] == 'Cook Islands', '2015 Population'] =  17459
df_pop_merge.loc[df_pop_merge['Country'] == 'Eritrea', '2015 Population'] =  4846976
df_pop_merge.loc[df_pop_merge['Country'] == 'Saint Kitts', '2015 Population'] =  54821
df_pop_mergena = df_pop_merge[df_pop_merge['2015 Population'].isnull()]
print(df_pop_mergena.groupby(['NOC', 'Country'])['NOC'].count())
#Drop the 106 observations not associated with a country
df_pop_merge = df_pop_merge.dropna(how = 'any')

from pandas.plotting import scatter_matrix
def calculate_bmi(df):
    return (df[1]/(df[0] * df[0])) * 10000

df_pop_merge['BMI'] = df_pop_merge[['Height', 'Weight']].apply(calculate_bmi, axis=1)
df_pop = df_pop_merge.groupby(['Country'])['2015 Population'].mean().reset_index()
df_pop['Population_Prop'] = df_pop['2015 Population'].transform(lambda x: x / (x.sum()))
df_pop_merge = pd.merge(left = df_pop_merge, right = df_pop, how = 'left', on=['Country'], 
                        left_index=False)
df_pop_merge = df_pop_merge[['Country', 'Sex', 'Age', 'Height', 'Weight', 'Year', 'Season', 'Sport', 'Event', 'BMI', 
                             'Population_Prop', 'Medal']]
df_pop_merge.to_csv('C:/Users/william.thigpen/Documents/olympics/data/pop_merge.csv')

#Change Sex, Season, and Medal to numeric factors
df_pop_merge.loc[df_pop_merge['Medal'] == 'No Medal', 'Medal'] =  0
df_pop_merge.loc[df_pop_merge['Medal'] == 'Bronze', 'Medal'] =  1
df_pop_merge.loc[df_pop_merge['Medal'] == 'Silver', 'Medal'] =  1
df_pop_merge.loc[df_pop_merge['Medal'] == 'Gold', 'Medal'] =  1
df_pop_merge.loc[df_pop_merge['Season'] == 'Summer', 'Season'] =  1
df_pop_merge.loc[df_pop_merge['Season'] == 'Winter', 'Season'] =  0
df_pop_merge.loc[df_pop_merge['Sex'] == 'M', 'Sex'] =  1
df_pop_merge.loc[df_pop_merge['Sex'] == 'F', 'Sex'] =  0


y = df_pop_merge['Medal']
X = df_pop_merge.drop(['Medal'], axis = 1)

NOC  Country                    
IOA  Individual Olympic Athletes    94
ROT  Refugee Athlete                12
Name: NOC, dtype: int64


### Modeling and Evaluation

#### Section 1

- (10 pts) Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

---

#### Section 2

- (10pts) Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

---

#### Section 3

- (20pts) Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

---

#### Section 4

- (10pts) Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

---

#### Section 5

- (10pts) Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.

---

#### Section 6

- (10pts) Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

---

### Deployment

- (5pts) How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

---

### Exceptional Work

- (10pts) You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

---